# Gompertz Tree-Diameter Simulation Comparison

This notebook implements and compares two versions of the stochastic Gompertz model for simulating tree diameter growth over time.

## Mathematical Models

### Version 1: With $X_{k-2}$ term

For this version, the diameter at year $k$ depends on the average of the two previous years:

$$X_k = D^{(1-e^{-r})} \cdot \left(\frac{X_{k-1} + X_{k-2}}{2}\right)^{(e^{-r})} \cdot \varepsilon_{k-1}, \quad k \geq 2$$

where:
- $X_0 = 15$ cm (fixed)
- $X_1 = 17$ cm (fixed)
- $\varepsilon_k \sim \text{Exponential}(\lambda)$ i.i.d.

### Version 2: With $X_{k-1}$ term only

For this simplified version, the diameter at year $k$ depends only on the previous year:

$$X_k = D^{(1-e^{-r})} \cdot X_{k-1}^{(e^{-r})} \cdot \varepsilon_{k-1}, \quad k \geq 1$$

where:
- $X_0 = 15$ cm (fixed)
- $\varepsilon_k \sim \text{Exponential}(\lambda)$ i.i.d.

## 1. Import Libraries and Configure Academic Style

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Ensure output directory exists
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Configure academic LaTeX-style plotting
plt.rcParams.update({
    # Font settings for LaTeX-like appearance
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'font.size': 11,
    'axes.labelsize': 13,
    'axes.titlesize': 14,
    'legend.fontsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    
    # Figure settings
    'figure.figsize': (8, 5.5),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    
    # Line settings
    'lines.linewidth': 2.0,
    'lines.markersize': 0,  # No markers for academic style
    
    # Grid settings
    'grid.alpha': 0.3,
    
    # Mathtext settings
    'mathtext.fontset': 'stix',
})

print("Libraries imported and academic style configured.")

## 2. Define Simulation Functions

In [ ]:
def simulate_gompertz_two_step(r, D, lam, n_years, X0=15.0, X1=17.0):
    """
    Simulate the Gompertz process with two-step dependency.

    Parameters
    ----------
    r : float
        Growth rate parameter.
    D : float
        Asymptotic diameter (maximum achievable diameter).
    lam : float
        Rate parameter for the exponential noise distribution.
    n_years : int
        Number of years to simulate.
    X0 : float, optional
        Initial diameter at year 0 (default: 15.0 cm).
    X1 : float, optional
        Diameter at year 1 (default: 17.0 cm).

    Returns
    -------
    np.ndarray
        Array of simulated diameters X_k for k = 0, 1, ..., n_years.
    """
    # Initialize the diameter array
    X = np.zeros(n_years + 1)
    X[0] = X0
    X[1] = X1

    # Pre-compute constant terms for efficiency
    exponent = np.exp(-r)
    constant_term = D ** (1 - exponent)

    # Generate all exponential noise variables at once
    epsilon = np.random.exponential(scale=1.0 / lam, size=n_years)

    # Simulate the process for k >= 2
    for k in range(2, n_years + 1):
        avg_previous = (X[k - 1] + X[k - 2]) / 2.0
        X[k] = constant_term * (avg_previous ** exponent) * epsilon[k - 1]

    return X


def simulate_gompertz_one_step(r, D, lam, n_years, X0=15.0):
    """
    Simulate the Gompertz process with one-step dependency.

    Parameters
    ----------
    r : float
        Growth rate parameter.
    D : float
        Asymptotic diameter (maximum achievable diameter).
    lam : float
        Rate parameter for the exponential noise distribution.
    n_years : int
        Number of years to simulate.
    X0 : float, optional
        Initial diameter at year 0 (default: 15.0 cm).

    Returns
    -------
    np.ndarray
        Array of simulated diameters X_k for k = 0, 1, ..., n_years.
    """
    # Initialize the diameter array
    X = np.zeros(n_years + 1)
    X[0] = X0

    # Pre-compute constant terms for efficiency
    exponent = np.exp(-r)
    constant_term = D ** (1 - exponent)

    # Generate all exponential noise variables at once
    epsilon = np.random.exponential(scale=1.0 / lam, size=n_years)

    # Simulate the process for k >= 1
    for k in range(1, n_years + 1):
        X[k] = constant_term * (X[k - 1] ** exponent) * epsilon[k - 1]

    return X


print("Simulation functions defined.")

## 3. Run Simulations with Default Parameters

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Define simulation parameters
params = {
    'r': 0.3,      # Growth rate parameter
    'D': 100.0,    # Asymptotic diameter (cm)
    'lambda': 0.7, # Rate parameter for exponential noise
    'years': 50,   # Number of years to simulate
    'X0': 15.0,    # Initial diameter
    'X1': 17.0     # Second year diameter (for Xkminus2 version)
}

# Run simulations
print("Running simulations...")
print(f"Parameters: r = {params['r']}, D = {params['D']}, λ = {params['lambda']}, years = {params['years']}")

# Version with X_{k-2} term
X_two_step = simulate_gompertz_two_step(
    r=params['r'],
    D=params['D'],
    lam=params['lambda'],
    n_years=params['years'],
    X0=params['X0'],
    X1=params['X1']
)

# Version with X_{k-1} term only
X_one_step = simulate_gompertz_one_step(
    r=params['r'],
    D=params['D'],
    lam=params['lambda'],
    n_years=params['years'],
    X0=params['X0']
)

print("Simulations complete.")
print(f"\nX_kminus2: Initial = {X_two_step[0]:.2f} cm, Final = {X_two_step[-1]:.2f} cm")
print(f"X_kminus1: Initial = {X_one_step[0]:.2f} cm, Final = {X_one_step[-1]:.2f} cm")

## 4. Create Academic-Style Comparison Plot

In [ ]:
def create_academic_comparison_plot(X_two_step, X_one_step, params, output_dir="output"):
    """
    Create a publication-quality comparison plot with academic LaTeX style.

    Features:
    - Inward-facing tick marks on all four sides
    - LaTeX-style serif fonts
    - Clean colored lines without markers
    - Boxed legend in top-right corner
    """
    # Create figure
    fig, ax = plt.subplots(figsize=(8, 5.5))

    # Generate year indices
    years = np.arange(len(X_two_step))

    # Plot trajectories with academic style (no markers)
    line1, = ax.plot(years, X_two_step, color='#17becf', linewidth=2.0,
                     label=r'$X^{(2)}_k$ (with $X_{k-2}$ term)')
    line2, = ax.plot(years, X_one_step, color='#ff7f0e', linewidth=2.0,
                     label=r'$X^{(1)}_k$ (with $X_{k-1}$ term only)')

    # Add reference line for asymptotic diameter D
    ax.axhline(y=params['D'], color='#7f7f7f', linestyle='--', 
               linewidth=1.5, alpha=0.8,
               label=r'$D = ' + f'{params["D"]:.0f}' + r'$ cm')

    # Set inward-facing tick marks on all four sides
    ax.tick_params(axis='both', which='major', direction='in', 
                   length=6, width=1.0)
    ax.tick_params(axis='both', which='minor', direction='in',
                   length=3, width=0.8)

    # Set axis labels with LaTeX-style formatting
    ax.set_xlabel(r'$k$ (Year)', fontsize=13)
    ax.set_ylabel(r'$X_k$ (cm)', fontsize=13)

    # Set title
    ax.set_title(r'Gompertz Stochastic Process Comparison\n' +
                 r'$r = ' + f'{params["r"]}' + r'$, $\lambda = ' + 
                 f'{params["lambda"]}' + r'$',
                 fontsize=14, pad=10)

    # Configure legend with rectangular box (sharp corners)
    legend = ax.legend(loc='upper right', frameon=True, 
                       fancybox=False, edgecolor='black',
                       framealpha=1.0, borderpad=0.8,
                       labelspacing=0.5)
    
    # Set legend box properties for sharp rectangular appearance
    legend.get_frame().set_linewidth(1.0)
    legend.get_frame().set_edgecolor('black')

    # Add light grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)

    # Set axis limits with some padding
    ax.set_xlim(-1, len(years) + 1)
    y_min = min(X_two_step.min(), X_one_step.min()) * 0.95
    y_max = max(X_two_step.max(), X_one_step.max()) * 1.05
    ax.set_ylim(max(0, y_min), y_max)

    # Adjust layout
    plt.tight_layout()

    # Save as PNG
    png_filename = os.path.join(output_dir, "gompertz_comparison_academic.png")
    plt.savefig(png_filename, dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    print(f"PNG saved: {png_filename}")

    # Save as PDF
    pdf_filename = os.path.join(output_dir, "gompertz_comparison_academic.pdf")
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight',
                facecolor='white', edgecolor='none')
    print(f"PDF saved: {pdf_filename}")

    # Display plot
    plt.show()

    return fig, ax


# Create and save the comparison plot
fig, ax = create_academic_comparison_plot(X_two_step, X_one_step, params, output_dir)

## 5. Summary and Parameter Information

In [ ]:
# Print summary
print("=" * 60)
print("GOMPERTZ SIMULATION SUMMARY")
print("=" * 60)

print("\nSimulation Parameters:")
print(f"  Growth rate (r):         {params['r']}")
print(f"  Asymptotic diameter (D): {params['D']} cm")
print(f"  Noise rate (λ):          {params['lambda']}")
print(f"  Number of years:         {params['years']}")

print("\nInitial Conditions:")
print(f"  X₀: {params['X0']} cm (both versions)")
print(f"  X₁: {params['X1']} cm (Xkminus2 version only)")

print("\nFinal Diameter Values:")
print(f"  Xkminus2 version: {X_two_step[-1]:.4f} cm")
print(f"  Xkminus1 version: {X_one_step[-1]:.4f} cm")
print(f"  Asymptote D:      {params['D']} cm")

print("\nOutput Files:")
print(f"  PNG: {os.path.join(output_dir, 'gompertz_comparison_academic.png')}")
print(f"  PDF: {os.path.join(output_dir, 'gompertz_comparison_academic.pdf')}")

print("=" * 60)